# Construction du modèle de reconnaissance de caractère basé sur le dataset MIST

Dans ce notebook sera présenté la création du modèle qui permettra de **classifier** les lettres de l'alphabet de a-z et A-Z.

Pour ce faire nous allons utiliser le dataset d'images prétraités créés par le notebook [NIST-preprocessing](/notebooks/notebooks/character_recognition/NIST-preprocessing.ipynb)

Pour exécuter ce notebook, veillez à ce que le jeu de données soit bien sous la forme suivante.
- data
    - processed
         - NIST-dataset
            - train
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png
            - test_set
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png


## Importation des dépendances

In [1]:
import os
from random import shuffle
from string import ascii_lowercase, ascii_uppercase

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-03-03 17:34:01.753390: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-03 17:34:01.787501: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-03 17:34:01.787987: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 17:34:02.482155: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Sélection des devices

In [2]:
from tensorflow.python.client import device_lib

print("Num of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

Num of GPUs Available:  0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10852856440311595628
 xla_global_id: -1]

In [3]:
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

## Définition des paramètres du notebook

In [4]:
NIST_PROCESSED_PATH = "../../data/processed/NIST-dataset"
TRAIN_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "train", "nist_processed_train.csv")
TEST_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "test", "nist_processed_test.csv")
IMG_SIZE = 41

## Importation du dataset

Dans cette section, nous allons charger le dataset afin d'obtenir un train_data_set, train_label_set, ainsi qu'un test_data_set et un test_label_set

### Chargement du dataset depuis le disque

In [5]:
train_set_frame = pd.read_csv(TRAIN_SET_PATH)
test_set_frame = pd.read_csv(TEST_SET_PATH)

In [6]:
def get_y_original_shape(y_raw_set):
    x  = y_raw_set.shape
    return y_raw_set.reshape(x[0], 1)

def get_x_original_shape(x_raw_set):
    x, y = x_raw_set.shape
    return x_raw_set.reshape(x, IMG_SIZE, IMG_SIZE)

In [7]:
def get_y_and_x_set_from_dataframe(dataframe):
    y_raw_set = dataframe["label"].to_numpy(dtype=str)
    x_raw_set = test_set_frame.iloc[:, 2:].to_numpy(dtype=np.uint8)

    y_set = get_y_original_shape(y_raw_set)
    x_set = get_x_original_shape(x_raw_set)

    return y_set, x_set

In [8]:
y_train, x_train = get_y_and_x_set_from_dataframe(train_set_frame)
y_test, x_test = get_y_and_x_set_from_dataframe(test_set_frame)

print(y_train.shape)
print(x_train.shape)

(8454, 1)
(8454, 41, 41)


## Encodage des labels

On récupère tous les labels possibles

In [9]:
label_list = [char for char in ascii_lowercase + ascii_uppercase]
print(label_list[0:5])

['a', 'b', 'c', 'd', 'e']


On définit l'encoder à utiliser

In [10]:
encoder = LabelBinarizer()
encoder.fit(label_list)

LabelBinarizer()

On encode nos données au format OneShot

In [11]:
y_encoded_train = encoder.transform(y_train)
y_encoded_test = encoder.transform(y_test)
y_encoded_train.shape

(8454, 52)

## Création du modèle de classification

In [23]:
model = keras.Sequential([
    keras.Input(shape=(41,41)),
    layers.Flatten(input_shape=(41, 41)),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(52, activation="softmax"),
])

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["acc", "categorical_crossentropy"])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 1681)              0         
                                                                 
 dense_25 (Dense)            (None, 100)               168200    
                                                                 
 dense_26 (Dense)            (None, 100)               10100     
                                                                 
 dense_27 (Dense)            (None, 100)               10100     
                                                                 
 dense_28 (Dense)            (None, 100)               10100     
                                                                 
 dense_29 (Dense)            (None, 52)                5252      
                                                                 
Total params: 203,752
Trainable params: 203,752
Non-tr

## Entrainement du modèle

In [24]:
model.fit(x_train, y_encoded_train, epochs=20)

Epoch 1/20
265/265 [==============================] - 1s 1ms/step - loss: 3.4427 - acc: 0.1313 - categorical_crossentropy: 3.4427
Epoch 2/20
265/265 [==============================] - 0s 1ms/step - loss: 3.0211 - acc: 0.2434 - categorical_crossentropy: 3.0211
Epoch 3/20
265/265 [==============================] - 0s 1ms/step - loss: 2.6965 - acc: 0.3156 - categorical_crossentropy: 2.6965
Epoch 4/20
265/265 [==============================] - 0s 1ms/step - loss: 2.4445 - acc: 0.3669 - categorical_crossentropy: 2.4445
Epoch 5/20
265/265 [==============================] - 0s 2ms/step - loss: 2.2376 - acc: 0.4197 - categorical_crossentropy: 2.2376
Epoch 6/20
265/265 [==============================] - 0s 1ms/step - loss: 2.0927 - acc: 0.4495 - categorical_crossentropy: 2.0927
Epoch 7/20
265/265 [==============================] - 0s 1ms/step - loss: 1.9622 - acc: 0.4786 - categorical_crossentropy: 1.9622
Epoch 8/20
265/265 [==============================] - 0s 1ms/step - loss: 1.8468 - acc: 0.

## Test du modèle sur le jeu de test

In [25]:
model.evaluate(x_test, y_encoded_test, verbose=2)

265/265 - 0s - loss: 1.3399 - acc: 0.6162 - categorical_crossentropy: 1.3399 - 303ms/epoch - 1ms/step


[1.3399312496185303, 0.6161580085754395, 1.3399312496185303]